<a href="https://colab.research.google.com/github/trangq/text_to_sql_agent/blob/main/RAG_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
      python-dotenv \
      langchain \
      langchain_openai \
      langchain_community \
      langchain-huggingface \
      langchain-google-genai \
      streamlit \
      faiss-cpu \
      sentence-transformers \
      pypdf \
      docx2txt

In [ ]:
#Khai báo khóa API key
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"]  = userdata.get('GOOGLE_API_KEY')

# Xây dựng chatbot đơn giãn với RAG

## Xây dựng Vector database

In [ ]:
#Tải dữ liệu từ folder (.txt)
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
file_path = "/content/drive/MyDrive/CamNang_pdf/M01_Fundamentals of Python_Python_Co_Ban_CamNang_202309.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
print(docs[0].page_content[:200])

 
 
 
 
 
 
FUNDAMENTALS OF PYTHON 
(LẬP TRÌNH PYTHON CƠ BẢN) 
Thời gian: 5 tuần 
Thời lượng: 36 giờ, học trực �ếp trên máy 
Học phí: 2.500.000 đ 
1. MỤC TIÊU 
 Khóa học cung cấp cho học viên (HV) nh


In [ ]:
from langchain_text_splitters.character import  RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=400,
      chunk_overlap=200
)
doc_chunks = text_splitter.split_documents(docs)
#Embedding các chunks này và đưa vào vector store
embeddings = HuggingFaceEmbeddings()
vectordb = FAISS.from_documents(doc_chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Lưu vector store
vectordb.save_local("index")

## Xây dựng chain

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
retriever = vectordb.as_retriever(
        search_kwargs={
                  "k": 2, # sl kq truy vấn trả về
    }
    )
memory = ConversationBufferMemory(
        llm=llm,
        output_key="answer",
        memory_key="chat_history",
        return_messages=True
    )
chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        chain_type="map_reduce",
        memory=memory,
        verbose=True
    )

<ipython-input-7-afe5c07ad5a0>:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
memory.clear()
while True:
    user_input = input("user: ")
    if user_input.lower() == "exit":
        break

    response = chain({"question": user_input, "chat_history": memory.chat_memory.messages})
    assistant_response = response["answer"]

    print("assistant: ", assistant_response)

user:  Nội dung của khóa học lập trình python cơ bản ?


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
Science 
 Tất cả những ai muốn �m hiểu về lập trình Python để ứng dụng trong công việc, học tập và cuộc 
sống. 
 Điều kiện cần khi tham gia khóa học: HV có kiến thức �n học văn phòng và sử dụng Internet 
3. KẾT QUẢ ĐẠT ĐƯỢC 
Sau khi hoàn thành khóa học, học viên sẽ đạt được các kỹ năng: 
 Sử dụng các cấu trúc trong ngôn ngữ lập trình Python để xây dựng ứng dụng
Human:  Nội dung của khóa học lập trình python cơ bản ?
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
có thể xây dựng các ứng dụng đơn giản b

In [ ]:
memory.chat_memory.messages

[HumanMessage(content=' Nội dung của khóa học lập trình python cơ bản ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Khóa học dạy học viên sử dụng cấu trúc trong Python để xây dựng ứng dụng đơn giản.  Nó cũng rèn luyện kỹ năng lập trình, tư duy logic, và xây dựng nền tảng cơ bản trong Python để học về lập trình web, game, desktop, machine learning, và data science.  Tuy nhiên, nội dung khóa học không được mô tả đầy đủ.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Mục tiêu của nó là gì ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mục tiêu của khóa học là giúp học viên hiểu lập trình Python để ứng dụng trong công việc, học tập và cuộc sống;  sử dụng các cấu trúc Python để xây dựng ứng dụng; rèn luyện kỹ năng lập trình và tư duy logic; và xây dựng nền tảng vững chắc trong Python để học các kiến thức lập trình Web, Game, Desktop, Machine Learning, Data Science.\n', additional_kwargs={}, response_metadata={})]